In [2]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"/user/test")


with mlflow.start_run() as run:
    # Load the diabetes dataset.
    db = load_diabetes()
    X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

    # Create and train models.
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
    rf.fit(X_train, y_train)

    # Use the model to make predictions on the test dataset.
    predictions = rf.predict(X_test)

    mlflow.sklearn.log_model(rf, "model")

    print("Run ID: {}".format(run.info.run_id))

2024/10/16 18:02:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/16 18:02:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run dapper-shrimp-273 at: http://localhost:5000/#/experiments/2/runs/8c8ef8aced0741ceaaa0b1a2cce9f9c9.
2024/10/16 18:02:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/2.


Run ID: 8c8ef8aced0741ceaaa0b1a2cce9f9c9


In [1]:
import pandas as pd
import numpy as np
np.float_ = np.float64
from prophet import Prophet
from datetime import timedelta, datetime
import mlflow
from sklearn.metrics import mean_absolute_error,mean_squared_error
from mlflow.models import infer_signature
from pathlib import Path
from autots import AutoTS
import pickle

In [2]:
def transformation():
    df_con=pd.DataFrame()
    mil=[]
    LI=pd.ExcelFile("Cours-de-change-2018_2024.xlsx")
    for g in LI.sheet_names:    
        df=pd.read_excel("Cours-de-change-2018_2024.xlsx",sheet_name=g)
        df=df.dropna(axis=0,how='all')
        som = {}  # Initialisez le dictionnaire som sans aucune clé initialement
        cle = ["DEVISES", "MIN EUR", "CMP EUR", "MAX EUR", "MIN USD", "CMP USD", "MAX USD"]

        # Initialisez chaque clé dans cle avec une liste vide dans som
        for c in cle:
            som[c] = []

        for inde, val in df.iterrows():
            bor = None
            for col in df.columns:
                if val[col] in cle:  # Trouver la colonne contenant une valeur dans cle
                    bor = df.columns.get_loc(col) + 1  # Obtenir l'index de la colonne suivante
                    break
            
            if bor is not None:  # Si une valeur dans cle a été trouvée
                som[val[col]].extend(val.iloc[bor:].tolist())  # Ajoutez les valeurs des colonnes suivantes à la liste existante
      
        dt=pd.DataFrame(som)
        mil.append(dt)  
        dt=pd.concat(mil,axis=0)
    dt=dt[dt["DEVISES"].notna()]
    dt["DEVISES"]=pd.to_datetime(dt["DEVISES"])
    return dt

In [3]:
dt=transformation()
#df["Period"]=df["DEVISES"]

In [107]:
df =df.set_index("Period")

In [108]:
df=df["MIN EUR"]

In [110]:
df.index = df.index.normalize()


In [112]:
df.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 1635 entries, 2024-01-02 to 2018-12-28
Series name: MIN EUR
Non-Null Count  Dtype  
--------------  -----  
1635 non-null   float64
dtypes: float64(1)
memory usage: 25.5 KB


In [113]:
import pandas as pd
from sktime.forecasting.fbprophet import Prophet

In [114]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"/user/test")
from sktime.utils import mlflow_sktime 

with mlflow.start_run() as run:
    forecaster = Prophet( )
    
    forecaster.fit(df)
    
    # artifact_path = "model"  

    # model_info = mlflow_sktime.log_model(

    # sktime_model=forecaster,

    # artifact_path=artifact_path)  



2024/10/31 10:39:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-grouse-110 at: http://localhost:5000/#/experiments/3/runs/ce254e9dde934219a26dea28405600f3.
2024/10/31 10:39:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


TypeError: y must be in an sktime compatible format, of scitype Series, Panel, Hierarchical, for instance a pandas.DataFrame with sktime compatible time indices, or with MultiIndex and last(-1) level an sktime compatible time index. See the forecasting tutorial examples/01_forecasting.ipynb, or the data format tutorial examples/AA_datatypes_and_datasets.ipynb,If you think y is already in an sktime supported input format, run sktime.datatypes.check_raise(y, mtype) to diagnose the error, where mtype is the string of the type specification you want for y. Possible mtype specification strings are as follows: "For Series scitype: ['pd.Series', 'pd.DataFrame', 'np.ndarray']. , "For Panel scitype: ['pd-multiindex', 'nested_univ', 'numpy3D', 'df-list', 'pd-long']. , "For Hierarchical scitype: ['pd_multiindex_hier']. 

In [22]:
def model_pickle(df):
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(f"/test")
    df=df[["DEVISES","CMP EUR"]]
    df=df.rename(columns={"CMP EUR":"y","DEVISES":"ds"})
    x=int(len(df.index)*0.8)
    x_train=df.iloc[:x]
    y_test=df.iloc[x:]
    model=Prophet()
    model.fit(x_train)
    app=model.predict(y_test)
    #y_pre=app["yhat"]
    filename = "propht_model.pkl"
    pickle.dump(model, open(filename, "wb"))
    
    with mlflow.start_run():
        
        mlflow.log_artifacts("propht_model.pkl",artifact_path='./michel')
        mlflow.end_run()

In [82]:
from sktime.datasets import load_airline
from sktime.forecasting.trend import ProphetPiecewiseLinearTrendForecaster
from sktime.forecasting.base import ForecastingHorizon
from sktime.split import temporal_train_test_split
 

In [84]:
y =load_airline()
y

Period
1949-01    112.0
1949-02    118.0
1949-03    132.0
1949-04    129.0
1949-05    121.0
           ...  
1960-08    606.0
1960-09    508.0
1960-10    461.0
1960-11    390.0
1960-12    432.0
Freq: M, Name: Number of airline passengers, Length: 144, dtype: float64

In [81]:
y =load_airline().to_timestamp(freq='M')
y_train, y_test = temporal_train_test_split(y)
fh = ForecastingHorizon(y.index, is_relative=False)
forecaster =  ProphetPiecewiseLinearTrendForecaster() 
forecaster.fit(y_train) 
y_pred = forecaster.predict(fh)

Period
1949-01-31    112.0
1949-02-28    118.0
1949-03-31    132.0
1949-04-30    129.0
1949-05-31    121.0
              ...  
1960-08-31    606.0
1960-09-30    508.0
1960-10-31    461.0
1960-11-30    390.0
1960-12-31    432.0
Freq: M, Name: Number of airline passengers, Length: 144, dtype: float64

In [23]:
y=model_pickle(dt)

19:59:39 - cmdstanpy - INFO - Chain [1] start processing
19:59:40 - cmdstanpy - INFO - Chain [1] done processing
2024/10/31 19:59:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run gifted-squirrel-648 at: http://localhost:5000/#/experiments/8/runs/b08bff40b2a44d78b2659dde72a1ae20.
2024/10/31 19:59:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/8.


In [18]:
def modelisationss(filename, df):
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(f"/user/test")

    # Charger le modèle depuis le fichier
    with open(filename, 'rb') as f:
        loaded_model = pickle.load(f)

    with mlflow.start_run():
        # Créer une signature basée sur les données d'entrée et de sortie
        x_train = df.iloc[:int(len(df.index)*0.8)]
        y_test = df.iloc[int(len(df.index)*0.8):]
        
        signature = infer_signature(x_train)
        reg_model_name = "prophet"

        print("--")
        # Sauvegarder le modèle avec MLflow
        mlflow.log_artifact_local(filename, artifact_path="models/prophets")
        # mlflow.pyfunc.log_model(
        #     python_model=loaded_model,
        #     artifact_path="prophet",
        #     signature=signature,
        #     registered_model_name=reg_model_name
        # )
    
    return signature


In [19]:
z=modelisationss(x,y)

2024/10/30 16:42:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run legendary-doe-402 at: http://localhost:5000/#/experiments/3/runs/640cc27638fd4105856d59e550c9ede7.
2024/10/30 16:42:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


--


AttributeError: module 'mlflow' has no attribute 'log_artifact_local'

In [35]:
def modelisationss(filename,df):

    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(f"/user/test")
    loaded_model = pickle.load(open(filename, "rb"))
    with mlflow.start_run():
    # create a signature for the model based on the input and output data
        x=int(len(df.index)*0.8)
        x_train=df.iloc[:x]
        y_test=df.iloc[x:]

        signature = infer_signature(x_train)
        reg_model_name = "SklearnLinearRegression"
        print("--")
        mlflow.pyfunc.log_model(path=loaded_model
    )
    return signature


In [36]:
z=modelisationss(x,y)

2024/10/28 22:08:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run sedate-vole-193 at: http://localhost:5000/#/experiments/3/runs/9c4ac0753fd14604bd05918ad1d38f20.
2024/10/28 22:08:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


--


TypeError: log_model() got an unexpected keyword argument 'path'

In [6]:
metric_weighting = {
    'smape_weighting': 5,
    'mae_weighting': 2,
    'rmse_weighting': 2,
    'made_weighting': 0.5,
    'mage_weighting': 1,
    'mle_weighting': 0,
    'imle_weighting': 0,
    'spl_weighting': 3,
    'containment_weighting': 0,
    'contour_weighting': 1,
    'runtime_weighting': 0.05,
}


In [5]:
model_list = [
 'Theta'
]

In [6]:
model = AutoTS(
    forecast_length=3,
    frequency='infer',
    ensemble='simple',
    max_generations=5,
    num_validations=2,
    model_list=model_list

)

Using 1 cpus for n_jobs.


In [7]:
model = model.fit(dt, date_col='DEVISES', value_col='MIN EUR')

Data frequency is: None, used frequency is: None
Frequency is 'None'! Data frequency not recognized.
One or more series is 90% or more NaN in this train split
One or more series is 90% or more NaN in this test split
Model Number: 1 with model FBProphet in generation 0 of 5
Template Eval Error: ValueError('The number of derivatives at boundaries does not match: expected 2, got 0+0') in model 1 in generation 0: FBProphet
Model Number: 2 with model MultivariateRegression in generation 0 of 5


/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/metrics.py:574: RuntimeWarning: Mean of empty slice
  scaler = np.nanmean(np.abs(np.diff(df_train[-100:], axis=0)), axis=0)
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/metrics.py:575: RuntimeWarning: All-NaN slice encountered
  fill_val = np.nanmax(scaler)


Model Number: 3 with model MultivariateRegression in generation 0 of 5
Template Eval Error: ValueError("regression_type='User' but not future_regressor supplied.") in model 3 in generation 0: MultivariateRegression
Model Number: 4 with model MultivariateRegression in generation 0 of 5
Model Number: 5 with model FBProphet in generation 0 of 5
Model Number: 6 with model Theta in generation 0 of 5
Model Number: 7 with model FBProphet in generation 0 of 5


/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/tools/percentile.py:47: RuntimeWarning: All-NaN slice encountered
  max_val = np.nanmax(arr)


Model Number: 8 with model FBProphet in generation 0 of 5
Model Number: 9 with model MultivariateRegression in generation 0 of 5
Model Number: 10 with model Theta in generation 0 of 5


...

/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/metrics.py:574: RuntimeWarning: Mean of empty slice
  scaler = np.nanmean(np.abs(np.diff(df_train[-100:], axis=0)), axis=0)
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/metrics.py:575: RuntimeWarning: All-NaN slice encountered
  fill_val = np.nanmax(scaler)


1 - Ensemble with avg smape nan: 
Model Number: 2 of 6 with model Ensemble for Validation 1
2 - Ensemble with avg smape nan: 
Model Number: 3 of 6 with model Ensemble for Validation 1
3 - Ensemble with avg smape nan: 
Model Number: 4 of 6 with model Ensemble for Validation 1
4 - Ensemble with avg smape nan: 
Model Number: 5 of 6 with model Ensemble for Validation 1
5 - Ensemble with avg smape nan: 
Model Number: 6 of 6 with model Ensemble for Validation 1
Template Eval Error: ValueError('BestN failed, no component models available.') in model 6 in generation 0: Ensemble
Validation Round: 2
One or more series is 90% or more NaN in this train split
One or more series is 90% or more NaN in this test split
Model Number: 1 of 6 with model Ensemble for Validation 2


/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/metrics.py:574: RuntimeWarning: Mean of empty slice
  scaler = np.nanmean(np.abs(np.diff(df_train[-100:], axis=0)), axis=0)
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/metrics.py:575: RuntimeWarning: All-NaN slice encountered
  fill_val = np.nanmax(scaler)


1 - Ensemble with avg smape nan: 
Model Number: 2 of 6 with model Ensemble for Validation 2
2 - Ensemble with avg smape nan: 
Model Number: 3 of 6 with model Ensemble for Validation 2
3 - Ensemble with avg smape nan: 
Model Number: 4 of 6 with model Ensemble for Validation 2
4 - Ensemble with avg smape nan: 
Model Number: 5 of 6 with model Ensemble for Validation 2
5 - Ensemble with avg smape nan: 
Model Number: 6 of 6 with model Ensemble for Validation 2
Template Eval Error: ValueError('BestN failed, no component models available.') in model 6 in generation 0: Ensemble


/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/auto_model.py:2481: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  per_series.idxmin(axis=1).rename("lowest_series_mape_name"),
/home/michel/code_python/mts/mlflow-minio-postgres/venv/lib/python3.11/site-packages/autots/evaluator/auto_model.py:2483: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a

In [9]:
best_model=model.best_model_name
best_model

'Theta'

In [21]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"/user/test")

with mlflow.start_run() as run:
    # Load the diabetes dataset.
    
    best_model=model.best_model_name
    # Use the model to make predictions on the test dataset.
    #predictions = rf.predict(X_test)

    mlflow.statsmodels.log_model(model, "model")


2024/10/31 13:37:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run treasured-rook-691 at: http://localhost:5000/#/experiments/3/runs/250cff9919d442a9b7cf1ca0b8a7b535.
2024/10/31 13:37:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


AttributeError: 'AutoTS' object has no attribute 'save'

In [1]:
print(model)

NameError: name 'model' is not defined

In [23]:
best_model=model.best_model_name

In [24]:
best_model

'Theta'

In [10]:
model.model_list

{'ConstantNaive': 1,
 'LastValueNaive': 1,
 'AverageValueNaive': 1,
 'GLS': 1,
 'SeasonalNaive': 1,
 'GLM': 1,
 'ETS': 1,
 'FBProphet': 0.5,
 'UnobservedComponents': 0.6,
 'VAR': 1,
 'VECM': 1,
 'ARIMA': 0.3,
 'WindowRegression': 0.8,
 'DatepartRegression': 1,
 'MultivariateRegression': 0.4,
 'UnivariateMotif': 1,
 'MultivariateMotif': 1,
 'SectionalMotif': 1,
 'NVAR': 0.4,
 'Theta': 1,
 'ARDL': 1,
 'ARCH': 1,
 'MetricMotif': 1,
 'SeasonalityMotif': 1,
 'DMD': 0.3,
 'RRVAR': 0.8,
 'FFT': 0.8,
 'Cassandra': 0.8}

: 

In [20]:
prediction = model.predict()


In [21]:
prediction

Prediction object: 
Return .forecast, 
 .upper_forecast, 
 .lower_forecast 
 .model_parameters 
 .transformation_parameters

In [27]:
prediction.transformation_parameters

{'fillna': 'mean',
 'transformations': {'0': 'Detrend'},
 'transformation_params': {'0': {'model': 'Linear', 'phi': 1, 'window': 90}}}

In [42]:
from sktime.datasets import load_airline
import pandas as pd
from sktime.forecasting.fbprophet import Prophet

In [43]:
y = load_airline().to_timestamp(freq='M')

In [44]:
y.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 144 entries, 1949-01-31 to 1960-12-31
Freq: M
Series name: Number of airline passengers
Non-Null Count  Dtype  
--------------  -----  
144 non-null    float64
dtypes: float64(1)
memory usage: 2.2 KB


In [69]:
x=len(y)

In [ ]:
x=

In [62]:
forecaster = Prophet(  

    seasonality_mode='multiplicative',

    n_changepoints=int(len(y) / 12),

    add_country_holidays={'country_name': 'Germany'},

    yearly_seasonality=True)

In [63]:
forecaster.fit(y) 

09:33:52 - cmdstanpy - INFO - Chain [1] start processing
09:33:52 - cmdstanpy - INFO - Chain [1] done processing


Prophet(add_country_holidays={'country_name': 'Germany'}, n_changepoints=12,
        seasonality_mode='multiplicative', yearly_seasonality=True)

In [64]:
my_list = list(range(1, 101))

In [65]:
y_pred = forecaster.predict(fh=my_list)

In [67]:
y_pred

1961-01-31    447.506448
1961-02-28    434.083666
1961-03-31    491.428372
1961-04-30    494.616029
1961-05-31    503.752697
                 ...    
1968-12-31    734.463672
1969-01-31    753.573200
1969-02-28    729.036992
1969-03-31    822.956653
1969-04-30    825.999511
Name: Number of airline passengers, Length: 100, dtype: float64

In [68]:
from sktime.performance_metrics.forecasting import     mean_absolute_percentage_error

In [ ]:
mean_absolute_percentage_error(y_true, y_pred, symmetric=False)

In [7]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from pathlib import Path
import mlflow.sklearn

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"/user/test")

iris = load_iris()
X_train = iris.data[:, :4]
Y_train = iris.target

with mlflow.start_run() as run:
    
    model = KNeighborsClassifier()
    model.fit(X_train, Y_train)
    mlflow.sklearn.log_model(model,"model")

2024/10/30 14:33:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 14:33:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run victorious-toad-966 at: http://localhost:5000/#/experiments/3/runs/36a093c3f1364caebf76d5a7eb7820a8.
2024/10/30 14:33:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


In [2]:
from autots.models.model_list import model_lists

In [3]:
print(model_lists)

{'all': ['ConstantNaive', 'LastValueNaive', 'AverageValueNaive', 'GLS', 'GLM', 'ETS', 'ARIMA', 'FBProphet', 'RollingRegression', 'GluonTS', 'SeasonalNaive', 'UnobservedComponents', 'VECM', 'DynamicFactor', 'MotifSimulation', 'WindowRegression', 'VAR', 'DatepartRegression', 'UnivariateRegression', 'UnivariateMotif', 'MultivariateMotif', 'NVAR', 'MultivariateRegression', 'SectionalMotif', 'Theta', 'ARDL', 'NeuralProphet', 'DynamicFactorMQ', 'PytorchForecasting', 'ARCH', 'RRVAR', 'MAR', 'TMF', 'LATC', 'KalmanStateSpace', 'MetricMotif', 'Cassandra', 'SeasonalityMotif', 'MLEnsemble', 'PreprocessingRegression', 'FFT', 'BallTreeMultivariateMotif', 'TiDE', 'NeuralForecast', 'DMD'], 'default': {'ConstantNaive': 1, 'LastValueNaive': 1, 'AverageValueNaive': 1, 'GLS': 1, 'SeasonalNaive': 1, 'GLM': 1, 'ETS': 1, 'FBProphet': 0.5, 'UnobservedComponents': 0.6, 'VAR': 1, 'VECM': 1, 'ARIMA': 0.3, 'WindowRegression': 0.8, 'DatepartRegression': 1, 'MultivariateRegression': 0.4, 'UnivariateMotif': 1, 'Mult